# DATA MODELING & SQLITE

## Creazione del DataBase

In [177]:
# creazione del database con l'obiettivo di avere un modello dati relazionale interrogabile via SQL
os.makedirs("data", exist_ok=True)
conn = sqlite3.connect("data/retail_assortment.db")

# verifica
conn.execute("SELECT 1").fetchall()

[(1,)]

In [178]:
# creazione della tabella fact_sales (ogni riga rappresenta 1 prodotto venduto in 1 outlet)
fact_sales_cols = [
    "ProductID",
    "OutletID",
    "OutletSales",
    "EstimatedUnitsSold",
    "ProductVisibility"]
# la ProductVisibility la inseriamo (anche se non è un driver decisionale primario) per non perdere in futuro la 
# possibilità di fare test sulla visibilità

fact_sales = df[fact_sales_cols]
fact_sales.head(1)

,ProductID,OutletID,OutletSales,EstimatedUnitsSold,ProductVisibility
0,FDA15,OUT049,3735.14,14.95,0.016047


In [179]:
# scrittura nel DB
fact_sales.to_sql(
    "fact_sales",
    conn,
    if_exists="replace",
    index=False)

8523

In [180]:
# creazione delle tabelle dimensionali
cursor = conn.cursor()

# Dimensione prodotto
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_product (
    ProductID TEXT PRIMARY KEY,
    ProductType TEXT,
    PriceBand TEXT
)
""")

# Dimensione outlet
cursor.execute("""
CREATE TABLE IF NOT EXISTS dim_outlet (
    OutletID TEXT PRIMARY KEY,
    OutletType TEXT,
    OutletSize TEXT
)
""")

conn.commit()

In [181]:
# popolamento della dimensione prodotto e scrittura nel DB
dim_product = (
    df[['ProductID', 'ProductType', 'PriceBand']]
    .drop_duplicates()
)

dim_product.to_sql(
    'dim_product',
    conn,
    if_exists='replace',
    index=False
)

1559

In [182]:
# popolamento della dimensione outlet e scrittura nel DB
dim_outlet = (
    df[['OutletID', 'OutletType', 'OutletSize']]
    .drop_duplicates()
)

dim_outlet.to_sql(
    'dim_outlet',
    conn,
    if_exists='replace',
    index=False
)

10

In [183]:
# verifico che i record caricati coincidano con la lunghezza delle tabelle 
# fact_tables ha 8523 righe come il df originale, dim_product ne ha 1559 come i productID unici e
# dim_outlet ne ha 10 come il numero di outletID unici

## Query di business

In [184]:
# posso procedere con le query SQL
# query che mostra le prime 10 righe di fact_sales (a termini di validazione schema)
query_head10_sales = """
SELECT
    *
FROM fact_sales 
LIMIT 10
"""
pd.read_sql(query_head10_sales, conn)

,ProductID,OutletID,OutletSales,EstimatedUnitsSold,ProductVisibility
0,FDA15,OUT049,3735.14,14.95,0.016047
1,DRC01,OUT018,443.42,9.19,0.019278
2,FDN15,OUT049,2097.27,14.81,0.016760
3,FDX07,OUT010,732.38,4.02,0.000000
4,NCD19,OUT013,994.71,18.47,0.000000
5,FDP36,OUT018,556.61,10.83,0.000000
6,FDO10,OUT013,343.55,5.96,0.012741
7,FDP10,OUT027,4022.76,37.33,0.127470
8,FDH17,OUT045,1076.60,11.10,0.016687
9,FDU28,OUT017,4710.54,25.08,0.094450


In [185]:
# Fatturato per categoria prodotto
query_ft_cat = """
SELECT p.ProductType,
    SUM(f.OutletSales) AS TotalSales
FROM fact_sales f
JOIN dim_product p ON f.ProductID = p.ProductID
GROUP BY p.ProductType
ORDER BY TotalSales DESC
"""
pd.read_sql(query_ft_cat, conn)

,ProductType,TotalSales
0,Fruits and Vegetables,2820059.87
1,Snack Foods,2732786.09
2,Household,2055493.85
3,Frozen Foods,1825734.96
4,Dairy,1522594.03
5,Canned,1444151.53
6,Baking Goods,1265525.36
7,Health and Hygiene,1045200.08
8,Meat,917565.48
9,Soft Drinks,892897.63


In [186]:
# produttività media SKU per outlet
query_prod_SKU_outlet="""
SELECT OutletID, sum(OutletSales) AS TotalSales, COUNT(DISTINCT ProductID) AS Num_SKU, 
    sum(OutletSales)/COUNT(DISTINCT ProductID) AS Sales_per_SKU
FROM fact_sales
GROUP BY OutletID"""
pd.read_sql(query_prod_SKU_outlet,conn)

,OutletID,TotalSales,Num_SKU,Sales_per_SKU
0,OUT010,188340.12,555,339.351568
1,OUT013,2142663.55,932,2298.995225
2,OUT017,2167465.48,926,2340.675464
3,OUT018,1851822.81,928,1995.498718
4,OUT019,179694.10,528,340.329735
5,OUT027,3453925.90,935,3694.038396
6,OUT035,2268122.99,930,2438.841925
7,OUT045,2036725.52,929,2192.384844
8,OUT046,2118395.22,930,2277.844323
9,OUT049,2183969.83,930,2348.354656


In [187]:
# top 10 SKU per fatturato
query_fatt_prod = """
SELECT
    ProductID,
    sum(OutletSales) as FATT_PROD
FROM fact_sales 
GROUP BY ProductID
ORDER BY FATT_PROD DESC
LIMIT 10
"""
pd.read_sql(query_fatt_prod, conn)

,ProductID,FATT_PROD
0,FDY55,42661.81
1,FDA15,41584.54
2,FDZ20,40185.02
3,FDF05,36555.75
4,FDA04,35741.49
5,FDK03,34843.98
6,NCQ06,34680.20
7,NCQ53,34508.41
8,FDJ55,33531.01
9,FDD44,32723.41


In [188]:
# FATTURATO PER OUTLET
query_ft_outlet="""
SELECT o.OutletID,
        sum(f.OutletSales) as FT_TOT
FROM fact_sales f
JOIN dim_outlet o ON o.OutletID=f.OutletID
GROUP BY o.OutletID
ORDER BY FT_TOT DESC
"""
pd.read_sql(query_ft_outlet,conn)

,OutletID,FT_TOT
0,OUT027,3453925.90
1,OUT035,2268122.99
2,OUT049,2183969.83
3,OUT017,2167465.48
4,OUT013,2142663.55
5,OUT046,2118395.22
6,OUT045,2036725.52
7,OUT018,1851822.81
8,OUT010,188340.12
9,OUT019,179694.10


In [189]:
# Fatturato per PriceBand
ft_PriceBand="""
SELECT p.PriceBand, sum(f.OutletSales) AS FT_TOT
FROM fact_sales f
JOIN dim_product p ON p.ProductID=f.ProductID
GROUP BY p.PriceBand
ORDER BY FT_TOT desc"""
pd.read_sql(ft_PriceBand,conn)

,PriceBand,FT_TOT
0,High,7326172.88
1,Mid-High,5536230.15
2,Mid-Low,3703573.34
3,Low,2025149.15


In [190]:
# SKU con fatt < 1000
worst_ft_SKU="""
SELECT ProductID, sum(OutletSales) AS FT_TOT
FROM fact_sales
GROUP BY ProductID
HAVING sum(OutletSales) < 1000
ORDER BY FT_TOT ASC"""
pd.read_sql(worst_ft_SKU,conn)

,ProductID,FT_TOT
0,FDQ60,120.51
1,NCR42,332.90
2,FDY43,673.79
3,FDF38,771.66
4,FDQ04,784.31


In [191]:
# categorie di prodotto con maggior ft medio per SKU
top_cat_ftm_SKU="""
SELECT p.ProductType, sum(f.OutletSales) AS FT_TOT, COUNT(DISTINCT f.ProductID) AS NUM_SKU, 
    sum(f.OutletSales)/COUNT(DISTINCT f.ProductID) AS FT_MEDIOxSKU
FROM fact_sales f
JOIN dim_product p ON f.ProductID=p.ProductID
GROUP BY p.ProductType
ORDER BY FT_MEDIOxSKU desc"""
pd.read_sql(top_cat_ftm_SKU,conn)

,ProductType,FT_TOT,NUM_SKU,FT_MEDIOxSKU
0,Seafood,148868.17,10,14886.817000
1,Fruits and Vegetables,2820059.87,220,12818.453955
2,Snack Foods,2732786.09,220,12421.754955
3,Breads,553237.21,45,12294.160222
4,Dairy,1522594.03,125,12180.752240
5,Household,2055493.85,170,12091.140294
6,Canned,1444151.53,120,12034.596083
7,Frozen Foods,1825734.96,155,11778.935226
8,Starchy Foods,351401.26,30,11713.375333
9,Breakfast,232298.98,20,11614.949000


# DASHBOARD (TABLEAU PUBLIC)

In [192]:
# per operare con Tableau Public ho bisogno di importare le tabelle in file supportati, come ad esempio .csv
# Recupero i nomi di tutte le tabelle nel database
tabelle = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

In [193]:
tabelle

,name
0,fact_sales
1,dim_product
2,dim_outlet


In [194]:
for nome_tabella in tabelle['name']:
    df = pd.read_sql_query(f"SELECT * FROM {nome_tabella}", conn)
    df.to_csv(f"{nome_tabella}.csv", index=False)
    print(f"Esportata tabella: {nome_tabella}")

Esportata tabella: fact_sales
Esportata tabella: dim_product
Esportata tabella: dim_outlet
